###### Content under Creative Commons Attribution license CC-BY 4.0, code under BSD 3-Clause License © 2018 D. Koehn, notebook style sheet by L.A. Barba, N.C. Clementi

###### Translated by Huizhe Di @ SCSIO

# 各向同性的线弹性介质

在上一节中，我们推导了波在一般各向异性线弹性介质中的传播运动方程。这里，我们将推导各向同性线弹性情况。

## 各向同性线弹性介质的应力-应变关系

一般各向异性线弹性介质的运动方程包含动量守恒:  

\begin{equation}
\rho \frac{\partial^2 u_i}{\partial t^2} = \sum_{j=1}^{3} \frac{\partial}{\partial x_j} \sigma_{ij} + f_i
\end{equation}

和应力-应变关系

\begin{equation}
\sigma_{ij} = \sum_{k=1}^{3}\sum_{l=1}^{3} c_{ijkl}\; \epsilon_{kl}
\end{equation}

为了引入各向同性，我们只需要修改弹性张量 $c_{ijkl}$, 因此方程(1)是不受影响的. 在各向同性体中，只有2个弹性常数是独立的，刚度张量 $c_{ijkl}$ 表现为这种形式 (Hodge 1961, Jeffreys 1969):



\begin{equation}
c_{ijkl}=\delta_{ij} \delta_{kl} \lambda + (\delta_{ik} \delta_{jl} + \delta_{il} \delta_{jk}) \mu
\end{equation}

这里

\begin{align}
\lambda\; \text{and}\; \mu:\; &\text{Lamé parameters} \nonumber \\ 
\delta_{ij}=
\begin{cases}
1 & \rm{\text{if } i = j}\\
0 & \rm{\text{if } i \ne j}
\end{cases} : \;&\text{Kronecker's delta} \nonumber
\end{align}

为了计算各向同性线弹性介质的应力-应变关系，我们必须将式(3)代入式(2)。在课堂上我们花了一些时间手工计算$\sigma_{13}$. 另外，我们也可以使用符号数学库**SymPy**.

In [1]:
# 输入SymPy
# 可使用“conda install sympy“或者”pip install sympy“安装
from sympy import *

In [2]:
# 测试KroneckerDelta函数
KroneckerDelta(1,2)

0

In [3]:
# 估计各向同性线弹性介质的应力应变关系
# using given indices i,j
def stress_strain_iso(i,j):
    
    # shift indices by -1
    i-=1
    j-=1
    
    # define cijkl for isotropic medium
    c_1 = KroneckerDelta(i,j)*KroneckerDelta(k,l)
    c_2 = KroneckerDelta(i,k)*KroneckerDelta(j,l)
    c_3 = KroneckerDelta(i,l)*KroneckerDelta(j,k)
    cijkl = c_1 * lam + (c_2 + c_3) * mu

    # sum over l-index
    sum_l = Sum(cijkl*e[k,l], (l, 0, 2)).doit()

    # sum over k-index to estimate stress
    stress = Sum(sum_l, (k, 0, 2)).doit()
    
    return stress

In [4]:
# define indices
i, j, k, l = symbols('i j k l', integer=True)

# define components of the deformation tensor as symbols
e11, e12, e13, e22, e23, e33 = symbols('e11 e12 e13 e22 e23 e33')

# define material parameters
lam, mu = symbols('lambda mu')

# define deformation tensor
e = Matrix([[e11,e12,e13], [e12,e22,e23], [e13,e23,e33]])

In [5]:
# stress-strain relation
stress = stress_strain_iso(1,3)
print(latex(stress))

2 e_{13} \mu


这是对**SymPy**计算能力的一个很好的介绍. 然而，我们不急于通过显式计算每个应力分量的Kronecker Deltas的和来评估各向同性介质的应力-应变关系, 让我们再仔细看看，我们必须评估什么:


\begin{equation}
\sigma_{ij} = \sum_{k=1}^{3}\sum_{l=1}^{3} \{\delta_{ij} \delta_{kl} \lambda + (\delta_{ik} \delta_{jl} + \delta_{il} \delta_{jk}) \mu\} \epsilon_{kl} \nonumber
\end{equation}

首先，我们重写了这个关系

\begin{equation}
\sigma_{ij} = \sum_{k=1}^{3}\biggl\{\sum_{l=1}^{3} \delta_{ij} \delta_{kl} \epsilon_{kl} \lambda + \mu \sum_{l=1}^{3} \delta_{ik} \delta_{jl} \epsilon_{kl} + \mu \sum_{l=1}^{3} \delta_{il} \delta_{jk} \epsilon_{kl} \biggr\}\nonumber
\end{equation}

求所有l指标的和. 式子 $\delta_{ij} \delta_{kl} \epsilon_{kl} \lambda$ 仅在$l=k$时非0, 因此我们得到:

\begin{equation}
\sigma_{ij} = \sum_{k=1}^{3}\biggl\{\delta_{ij} \epsilon_{kk} \lambda + \mu \sum_{l=1}^{3} \delta_{ik} \delta_{jl} \epsilon_{kl} + \mu \sum_{l=1}^{3} \delta_{il} \delta_{jk} \epsilon_{kl} \biggr\}\nonumber
\end{equation}

式子仅在$\delta_{ik} \delta_{jl} \epsilon_{kl}$仅在$l=j$时非0:

\begin{equation}
\sigma_{ij} = \sum_{k=1}^{3}\biggl\{\delta_{ij} \epsilon_{kk} \lambda + \mu \delta_{ik} \epsilon_{kj} + \mu \sum_{l=1}^{3} \delta_{il} \delta_{jk} \epsilon_{kl} \biggr\}\nonumber
\end{equation}

这个术语的情况正好相反 $\delta_{il} \delta_{jk} \epsilon_{kl}$, 其中仅有$l=i$对求和有贡献:

\begin{equation}
\sigma_{ij} = \sum_{k=1}^{3}\biggl\{\delta_{ij} \epsilon_{kk} \lambda + \mu \delta_{ik} \epsilon_{kj} + \mu \delta_{jk} \epsilon_{ki} \biggr\}\nonumber
\end{equation}

接下来，我们可以求k指标的和, 这里 $\mu \delta_{ik} \epsilon_{kj}$ 非0在 $k=i$ 和 $\mu \delta_{jk} \epsilon_{ki}$ 非0在 $k=j$, 导致 

\begin{equation}
\sigma_{ij} = \delta_{ij} \lambda \biggl\{\sum_{k=1}^{3} \epsilon_{kk} \biggr\} + \mu  \epsilon_{ij} + \mu \epsilon_{ji} \nonumber
\end{equation}

由于应变张量的对称性 $\epsilon_{ij} = \epsilon_{ji}$, 我们得到

\begin{equation}
\sigma_{ij} = \delta_{ij} \lambda \biggl\{\sum_{k=1}^{3} \epsilon_{kk} \biggr\} + 2 \mu \epsilon_{ij} \nonumber
\end{equation}

最后，一般的“各向同性线弹性介质的应力-应变关系”为:

\begin{equation}
\sigma_{ij}=\lambda \theta \delta_{ij} + 2 \mu \epsilon_{ij} \nonumber
\end{equation}

随着三次膨胀

\begin{equation}
\theta = tr(\epsilon_{ij}) =  \epsilon_{11} + \epsilon_{22} + \epsilon_{33} = \frac{\partial u_1}{\partial x_1} + \frac{\partial u_2}{\partial x_2} + \frac{\partial u_3}{\partial x_3} \nonumber
\end{equation}


## 三维各向同性线弹性介质的运动方程

写出各向同性线弹性介质的动量守恒和应力应变关系，得到:

\begin{align}
\rho\frac{\partial^2 u_1}{\partial t^2} &= \frac{\partial \sigma_{11}}{\partial x_1} + \frac{\partial \sigma_{12}}{\partial x_2} + \frac{\partial \sigma_{13}}{\partial x_3} + f_1 \nonumber \\
\rho\frac{\partial^2 u_2}{\partial t^2} &= \frac{\partial \sigma_{21}}{\partial x_1} + \frac{\partial \sigma_{22}}{\partial x_2} + \frac{\partial \sigma_{23}}{\partial x_3} + f_2 \nonumber \\
\rho\frac{\partial^2 u_3}{\partial t^2} &= \frac{\partial \sigma_{31}}{\partial x_1} + \frac{\partial \sigma_{32}}{\partial x_2} + \frac{\partial \sigma_{33}}{\partial x_3} + f_3 \nonumber \\
\sigma_{11} &= \lambda(\epsilon_{11}+\epsilon_{22}+\epsilon_{33}) + 2 \mu \epsilon_{11}\nonumber \\
\sigma_{22} &= \lambda(\epsilon_{11}+\epsilon_{22}+\epsilon_{33}) + 2 \mu \epsilon_{22}\nonumber \\
\sigma_{33} &= \lambda(\epsilon_{11}+\epsilon_{22}+\epsilon_{33}) + 2 \mu \epsilon_{33}\nonumber \\
\sigma_{12} &= 2 \mu \epsilon_{12}\nonumber \\
\sigma_{13} &= 2 \mu \epsilon_{13}\nonumber \\
\sigma_{23} &= 2 \mu \epsilon_{23}\nonumber \\
\end{align}


## 我们学习到:

* 推导各向同性线弹性介质应力应变关系的3种不同方法:

  1. 手工的，通过直接计算应力张量分量
  2. 通过数学符号库 **SymPy**
  3. 对应力-应变关系的一般评价，忽略所有非贡献项
* 三维各向同性线弹性介质的运动方程

## 参考文献

1. Hodge, P.G. (1961) "[On Isotropic Cartesian Tensors](https://www.jstor.org/stable/2311997?seq=1#page_scan_tab_contents)", The American Mathematical Monthly, 68(8), 793-95.

2. Jeffreys, H. (1969) "[Cartesian Tensors](https://archive.org/details/CartesianTensors)", Cambridge University Press
